In [1]:
# Import our dependencies.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
# Import the kerastuner library
import keras_tuner as kt


In [2]:
#  Import and read the datasets.csv
import pandas as pd 
crime_df = pd.read_csv("../cleaned_data_2015_2018/all_data_merged_cleaned_2015_2018.csv", index_col=False)
crime_df

,event_unique_id,crime,occ_year,occ_month,occ_day,occ_dow,occ_hour,premises_type,hood_140,neighbourhood_140,long_wgs84,lat_wgs84,date
0,GO-20155154,assault,2015,January,1,Thursday,23,House,115,Mount Dennis (115),-79.504668,43.693238,2015-01-01
1,GO-20151233,assault,2015,January,1,Thursday,3,Commercial,77,Waterfront Communities-The Island (77),-79.392855,43.647315,2015-01-01
2,GO-2015862,assault,2015,January,1,Thursday,2,Commercial,77,Waterfront Communities-The Island (77),-79.387700,43.649776,2015-01-01
3,GO-2015182,assault,2015,January,1,Thursday,0,Commercial,1,West Humber-Clairville (1),-79.595562,43.686751,2015-01-01
4,GO-2015276,assault,2015,January,1,Thursday,0,Outside,77,Waterfront Communities-The Island (77),-79.400096,43.645835,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166507,GO-20181887731,homicide,2018,October,12,Friday,0,NaN,75,Church-Yonge Corridor (75),-79.376699,43.656958,2018-10-12
166508,GO-20181923388,homicide,2018,October,18,Thursday,0,NaN,120,Clairlea-Birchmount (120),-79.286292,43.699070,2018-10-18
166509,GO-20181939006,homicide,2018,October,20,Saturday,0,NaN,124,Kennedy Park (124),-79.264551,43.732765,2018-10-20
166510,GO-20182113063,homicide,2018,November,16,Friday,0,NaN,73,Moss Park (73),-79.369005,43.654614,2018-11-16


In [3]:
# Create new dataframe by counting occurences of certain crimes and grouping by date and day of the week (occ_dow)
# From MM script
daily_crime = crime_df.groupby(by=['date', "occ_dow"])["crime"].value_counts().unstack(fill_value=0)
daily_crime

,crime,assault,auto_theft,bicycle_theft,break_and_enter,homicide,robbery,shooting,theft_from_motor_vehicle,theft_over
date,occ_dow,,,,,,,,,
2015-01-01,Thursday,158,7,5,21,0,8,0,34,5
2015-01-02,Friday,34,9,4,18,0,6,0,34,2
2015-01-03,Saturday,37,10,0,4,0,8,0,21,3
2015-01-04,Sunday,44,5,1,11,0,5,0,33,1
2015-01-05,Monday,39,6,1,24,0,5,1,31,2
...,...,...,...,...,...,...,...,...,...,...
2018-12-27,Thursday,38,7,2,21,0,7,0,30,1
2018-12-28,Friday,35,13,2,21,0,12,0,31,1
2018-12-29,Saturday,37,10,3,26,0,7,0,21,4


In [4]:
# Rename columns for data clarity
#From MM script
target_columns = [f'{column}_count' for column in daily_crime.columns]

daily_crime.columns = target_columns
daily_crime

,,assault_count,auto_theft_count,bicycle_theft_count,break_and_enter_count,homicide_count,robbery_count,shooting_count,theft_from_motor_vehicle_count,theft_over_count
date,occ_dow,,,,,,,,,
2015-01-01,Thursday,158,7,5,21,0,8,0,34,5
2015-01-02,Friday,34,9,4,18,0,6,0,34,2
2015-01-03,Saturday,37,10,0,4,0,8,0,21,3
2015-01-04,Sunday,44,5,1,11,0,5,0,33,1
2015-01-05,Monday,39,6,1,24,0,5,1,31,2
...,...,...,...,...,...,...,...,...,...,...
2018-12-27,Thursday,38,7,2,21,0,7,0,30,1
2018-12-28,Friday,35,13,2,21,0,12,0,31,1
2018-12-29,Saturday,37,10,3,26,0,7,0,21,4


In [5]:
# Reset index to flatten the dataframe
daily_crime.reset_index(inplace=True)
daily_crime.columns

Index(['date', 'occ_dow', 'assault_count', 'auto_theft_count',
       'bicycle_theft_count', 'break_and_enter_count', 'homicide_count',
       'robbery_count', 'shooting_count', 'theft_from_motor_vehicle_count',
       'theft_over_count'],
      dtype='object')

In [6]:
#  Import and read the datasets.csv
import pandas as pd 
weather_df = pd.read_csv("../cleaned_data_2015_2018/toronto_daily_weather_2015_2018.csv", index_col=False)
weather_df

,date,max_temperature,min_temperature,max_relative_humidity,avg_relative_humidity,avg_pressure_sea,max_wind_speed,precipitation,rain,snow,snow_on_ground,daylight,avg_cloud_cover_8
0,2018-12-31,5.0,-2.5,94,79.5,100.78,21,13.6,13.6,0.0,0.0,8.98,4.5
1,2018-12-30,0.7,-6.6,88,78.0,102.03,24,1.2,0.0,1.6,2.0,8.97,5.0
2,2018-12-29,2.7,-7.0,80,68.0,101.86,34,0.0,0.0,0.0,0.0,8.95,4.5
3,2018-12-28,12.4,2.7,95,84.5,100.89,30,1.8,1.8,0.0,0.0,8.93,4.5
4,2018-12-27,2.8,-7.4,87,74.0,102.52,30,5.4,5.4,0.0,0.0,8.93,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2015-01-05,-5.8,-14.4,83,69.5,102.38,45,0.8,0.0,0.8,1.0,9.05,3.5
1457,2015-01-04,4.3,-5.9,98,82.5,100.69,43,6.8,6.4,0.4,0.0,9.02,6.0
1458,2015-01-03,1.5,-5.8,97,82.5,101.96,29,10.8,8.0,2.4,0.0,9.00,5.0
1459,2015-01-02,-1.0,-6.1,83,66.0,102.30,36,0.6,0.0,0.4,0.0,9.00,6.0


In [7]:
weather_df.columns

Index(['date', 'max_temperature', 'min_temperature', 'max_relative_humidity',
       'avg_relative_humidity', 'avg_pressure_sea', 'max_wind_speed',
       'precipitation', 'rain', 'snow', 'snow_on_ground', 'daylight',
       'avg_cloud_cover_8'],
      dtype='object')

In [10]:
# Drop the non-beneficial ID columns, from weather
# For optimization remove only 'EIN' and 'STATUS', keep 'NAME'
weather_df.drop(columns=['max_relative_humidity', 'rain', 'snow'], inplace=True)
weather_df.head()

,date,max_temperature,min_temperature,avg_relative_humidity,avg_pressure_sea,max_wind_speed,precipitation,snow_on_ground,daylight,avg_cloud_cover_8
0,2018-12-31,5.0,-2.5,79.5,100.78,21,13.6,0.0,8.98,4.5
1,2018-12-30,0.7,-6.6,78.0,102.03,24,1.2,2.0,8.97,5.0
2,2018-12-29,2.7,-7.0,68.0,101.86,34,0.0,0.0,8.95,4.5
3,2018-12-28,12.4,2.7,84.5,100.89,30,1.8,0.0,8.93,4.5
4,2018-12-27,2.8,-7.4,74.0,102.52,30,5.4,0.0,8.93,4.0


In [11]:
# Merge weather and crime dataframes
alldata_df = pd.merge(weather_df, daily_crime, on="date", how="left")

alldata_df

,date,max_temperature,min_temperature,avg_relative_humidity,avg_pressure_sea,max_wind_speed,precipitation,snow_on_ground,daylight,avg_cloud_cover_8,occ_dow,assault_count,auto_theft_count,bicycle_theft_count,break_and_enter_count,homicide_count,robbery_count,shooting_count,theft_from_motor_vehicle_count,theft_over_count
0,2018-12-31,5.0,-2.5,79.5,100.78,21,13.6,0.0,8.98,4.5,Monday,38,14,2,20,0,11,0,23,4
1,2018-12-30,0.7,-6.6,78.0,102.03,24,1.2,2.0,8.97,5.0,Sunday,43,8,3,22,0,7,0,22,3
2,2018-12-29,2.7,-7.0,68.0,101.86,34,0.0,0.0,8.95,4.5,Saturday,37,10,3,26,0,7,0,21,4
3,2018-12-28,12.4,2.7,84.5,100.89,30,1.8,0.0,8.93,4.5,Friday,35,13,2,21,0,12,0,31,1
4,2018-12-27,2.8,-7.4,74.0,102.52,30,5.4,0.0,8.93,4.0,Thursday,38,7,2,21,0,7,0,30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2015-01-05,-5.8,-14.4,69.5,102.38,45,0.8,1.0,9.05,3.5,Monday,39,6,1,24,0,5,1,31,2
1457,2015-01-04,4.3,-5.9,82.5,100.69,43,6.8,0.0,9.02,6.0,Sunday,44,5,1,11,0,5,0,33,1
1458,2015-01-03,1.5,-5.8,82.5,101.96,29,10.8,0.0,9.00,5.0,Saturday,37,10,0,4,0,8,0,21,3
1459,2015-01-02,-1.0,-6.1,66.0,102.30,36,0.6,0.0,9.00,6.0,Friday,34,9,4,18,0,6,0,34,2


In [12]:
alldata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   date                            1461 non-null   object 
 1   max_temperature                 1461 non-null   float64
 2   min_temperature                 1461 non-null   float64
 3   avg_relative_humidity           1461 non-null   float64
 4   avg_pressure_sea                1461 non-null   float64
 5   max_wind_speed                  1461 non-null   int64  
 6   precipitation                   1461 non-null   float64
 7   snow_on_ground                  1461 non-null   float64
 8   daylight                        1461 non-null   float64
 9   avg_cloud_cover_8               1461 non-null   float64
 10  occ_dow                         1461 non-null   object 
 11  assault_count                   1461 non-null   int64  
 12  auto_theft_count                14

In [13]:
# Convert date into Unix timestamp for NN model
import pandas as pd
from datetime import datetime
import numpy as np

alldata_df['date'] = pd.to_datetime(alldata_df['date'])
#alldata_df['unix_timestamp'] = alldata_df.date.astype(int) / 10**9
alldata_df['unix_timestamp'] = alldata_df.date.values.astype(np.int64) // 10**9
alldata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            1461 non-null   datetime64[ns]
 1   max_temperature                 1461 non-null   float64       
 2   min_temperature                 1461 non-null   float64       
 3   avg_relative_humidity           1461 non-null   float64       
 4   avg_pressure_sea                1461 non-null   float64       
 5   max_wind_speed                  1461 non-null   int64         
 6   precipitation                   1461 non-null   float64       
 7   snow_on_ground                  1461 non-null   float64       
 8   daylight                        1461 non-null   float64       
 9   avg_cloud_cover_8               1461 non-null   float64       
 10  occ_dow                         1461 non-null   object        
 11  assa

In [14]:
# Remove date column
alldata_df.drop(columns=["date"], axis=1, inplace=True)
alldata_df


,max_temperature,min_temperature,avg_relative_humidity,avg_pressure_sea,max_wind_speed,precipitation,snow_on_ground,daylight,avg_cloud_cover_8,occ_dow,assault_count,auto_theft_count,bicycle_theft_count,break_and_enter_count,homicide_count,robbery_count,shooting_count,theft_from_motor_vehicle_count,theft_over_count,unix_timestamp
0,5.0,-2.5,79.5,100.78,21,13.6,0.0,8.98,4.5,Monday,38,14,2,20,0,11,0,23,4,1546214400
1,0.7,-6.6,78.0,102.03,24,1.2,2.0,8.97,5.0,Sunday,43,8,3,22,0,7,0,22,3,1546128000
2,2.7,-7.0,68.0,101.86,34,0.0,0.0,8.95,4.5,Saturday,37,10,3,26,0,7,0,21,4,1546041600
3,12.4,2.7,84.5,100.89,30,1.8,0.0,8.93,4.5,Friday,35,13,2,21,0,12,0,31,1,1545955200
4,2.8,-7.4,74.0,102.52,30,5.4,0.0,8.93,4.0,Thursday,38,7,2,21,0,7,0,30,1,1545868800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,-5.8,-14.4,69.5,102.38,45,0.8,1.0,9.05,3.5,Monday,39,6,1,24,0,5,1,31,2,1420416000
1457,4.3,-5.9,82.5,100.69,43,6.8,0.0,9.02,6.0,Sunday,44,5,1,11,0,5,0,33,1,1420329600
1458,1.5,-5.8,82.5,101.96,29,10.8,0.0,9.00,5.0,Saturday,37,10,0,4,0,8,0,21,3,1420243200
1459,-1.0,-6.1,66.0,102.30,36,0.6,0.0,9.00,6.0,Friday,34,9,4,18,0,6,0,34,2,1420156800


#### PreProcessing


In [16]:
# Determine the number of unique values in each column.
unique =alldata_df.nunique()
unique

max_temperature                    434
min_temperature                    409
avg_relative_humidity              120
avg_pressure_sea                   323
max_wind_speed                      52
precipitation                      112
snow_on_ground                      22
daylight                           383
avg_cloud_cover_8                   16
occ_dow                              7
assault_count                       75
auto_theft_count                    26
bicycle_theft_count                 32
break_and_enter_count               37
homicide_count                       3
robbery_count                       21
shooting_count                       4
theft_from_motor_vehicle_count      42
theft_over_count                    13
unix_timestamp                    1461
dtype: int64

In [17]:
# # Look at APPLICATION_TYPE value counts for binning
# hood_counts =alldata_df.hood_140.value_counts()
# hood_counts

In [18]:
# # Visualize the value counts of APPLICATION_TYPE
# hood_counts.plot.density()

In [20]:
# Generate our categorical variable lists
data_cat = alldata_df.dtypes[alldata_df.dtypes == "object"].index.tolist()

data_cat

['occ_dow']

In [21]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(alldata_df[data_cat])) 

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(data_cat)
encode_df.head()

c:\Users\User\anaconda3\envs\mleng\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,occ_dow_Friday,occ_dow_Monday,occ_dow_Saturday,occ_dow_Sunday,occ_dow_Thursday,occ_dow_Tuesday,occ_dow_Wednesday
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
# Merge one-hot encoded features and drop the originals
alldata_df = alldata_df.merge(encode_df,left_index=True, right_index=True)
alldata_df = alldata_df.drop(data_cat,1)
alldata_df.head()

c:\Users\User\anaconda3\envs\mleng\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,max_temperature,min_temperature,avg_relative_humidity,avg_pressure_sea,max_wind_speed,precipitation,snow_on_ground,daylight,avg_cloud_cover_8,assault_count,...,theft_from_motor_vehicle_count,theft_over_count,unix_timestamp,occ_dow_Friday,occ_dow_Monday,occ_dow_Saturday,occ_dow_Sunday,occ_dow_Thursday,occ_dow_Tuesday,occ_dow_Wednesday
0,5.0,-2.5,79.5,100.78,21,13.6,0.0,8.98,4.5,38,...,23,4,1546214400,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.7,-6.6,78.0,102.03,24,1.2,2.0,8.97,5.0,43,...,22,3,1546128000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2.7,-7.0,68.0,101.86,34,0.0,0.0,8.95,4.5,37,...,21,4,1546041600,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,12.4,2.7,84.5,100.89,30,1.8,0.0,8.93,4.5,35,...,31,1,1545955200,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.8,-7.4,74.0,102.52,30,5.4,0.0,8.93,4.0,38,...,30,1,1545868800,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Select Scope of model

In [24]:
# Select only 1 crime type and drop the rest
alldata_df.drop(columns=['assault_count', 'auto_theft_count', 'break_and_enter_count', 'homicide_count', 'robbery_count', 'shooting_count'], inplace=True)
alldata_df.drop(columns=['theft_from_motor_vehicle_count','theft_over_count'], inplace=True)
alldata_df

,max_temperature,min_temperature,avg_relative_humidity,avg_pressure_sea,max_wind_speed,precipitation,snow_on_ground,daylight,avg_cloud_cover_8,bicycle_theft_count,unix_timestamp,occ_dow_Friday,occ_dow_Monday,occ_dow_Saturday,occ_dow_Sunday,occ_dow_Thursday,occ_dow_Tuesday,occ_dow_Wednesday
0,5.0,-2.5,79.5,100.78,21,13.6,0.0,8.98,4.5,2,1546214400,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.7,-6.6,78.0,102.03,24,1.2,2.0,8.97,5.0,3,1546128000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2.7,-7.0,68.0,101.86,34,0.0,0.0,8.95,4.5,3,1546041600,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,12.4,2.7,84.5,100.89,30,1.8,0.0,8.93,4.5,2,1545955200,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.8,-7.4,74.0,102.52,30,5.4,0.0,8.93,4.0,2,1545868800,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,-5.8,-14.4,69.5,102.38,45,0.8,1.0,9.05,3.5,1,1420416000,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1457,4.3,-5.9,82.5,100.69,43,6.8,0.0,9.02,6.0,1,1420329600,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1458,1.5,-5.8,82.5,101.96,29,10.8,0.0,9.00,5.0,0,1420243200,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1459,-1.0,-6.1,66.0,102.30,36,0.6,0.0,9.00,6.0,4,1420156800,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Split our preprocessed data into our features and target arrays
crime_type = 'bicycle_theft_count'
y = alldata_df[crime_type].values
X = alldata_df.drop([crime_type],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

c:\Users\User\anaconda3\envs\mleng\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Verify X_Train
X_train_scaled

array([[ 1.35129656,  1.59671693,  0.85178009, ...,  2.47192023,
        -0.39532645, -0.39068092],
       [ 1.33442317,  1.42593719,  0.00621826, ...,  2.47192023,
        -0.39532645, -0.39068092],
       [-0.23480208,  0.14508913,  0.98528985, ..., -0.4045438 ,
         2.52955503, -0.39068092],
       ...,
       [-1.04472479, -1.65758592, -1.1953696 , ..., -0.4045438 ,
        -0.39532645,  2.55963359],
       [ 1.10663241,  1.0084756 ,  0.4512508 , ..., -0.4045438 ,
        -0.39532645, -0.39068092],
       [-0.63132674, -0.24390917,  2.36489073, ..., -0.4045438 ,
        -0.39532645, -0.39068092]])

#### Build single model

In [29]:
# Check number of inputs
len(X_train[0])

17

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

number_input_features = len(X_train[0])
hidden_nodes_layer1 = number_input_features * 6
hidden_nodes_layer2 = number_input_features * 6
hidden_nodes_layer3 = number_input_features * 6
hidden_nodes_layer4 = number_input_features * 3
hidden_nodes_layer5 = number_input_features * 3
hidden_nodes_layer6 = number_input_features * 3

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="tanh", input_dim=number_input_features))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Fifth hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))

# Sixth hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 102)               1836      
                                                                 
 dense_1 (Dense)             (None, 102)               10506     
                                                                 
 dense_2 (Dense)             (None, 102)               10506     
                                                                 
 dense_3 (Dense)             (None, 51)                5253      
                                                                 
 dense_4 (Dense)             (None, 51)                2652      
                                                                 
 dense_5 (Dense)             (None, 51)                2652      
                                                                 
 dense_6 (Dense)             (None, 1)                 5

In [31]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["mae"])
#nn_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae"])

In [32]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
35/35 [==============================] - 2s 2ms/step - loss: -27.6625 - accuracy: 0.0365
Epoch 2/100
35/35 [==============================] - 0s 2ms/step - loss: -76.4343 - accuracy: 0.0630
Epoch 3/100
35/35 [==============================] - 0s 2ms/step - loss: -96.7448 - accuracy: 0.0667
Epoch 4/100
35/35 [==============================] - 0s 2ms/step - loss: -112.9208 - accuracy: 0.0676
Epoch 5/100
35/35 [==============================] - 0s 2ms/step - loss: -127.5629 - accuracy: 0.0658
Epoch 6/100
35/35 [==============================] - 0s 2ms/step - loss: -142.8377 - accuracy: 0.0685
Epoch 7/100
35/35 [==============================] - 0s 2ms/step - loss: -158.5298 - accuracy: 0.0703
Epoch 8/100
35/35 [==============================] - 0s 2ms/step - loss: -173.6742 - accuracy: 0.0676
Epoch 9/100
35/35 [==============================] - 0s 2ms/step - loss: -188.0398 - accuracy: 0.0676
Epoch 10/100
35/35 [==============================] - 0s 2ms/step - loss: -202.2669 -

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: -1.4785e+03 - accuracy: 0.0492 - 208ms/epoch - 17ms/step
Loss: -1478.4639892578125, Accuracy: 0.04918032884597778


In [34]:
# Save Model  
nn_model.save("Crime_singleLayer.h5")